<a href="https://colab.research.google.com/github/Runshi-Yang/JSC270_HW4_2022_Runshi/blob/main/Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.stem.porter import *
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
train_data = pd.read_csv('https://raw.githubusercontent.com/jsAiyaya/JSC270_Data_Science_1/main/covid-tweets-train.csv', encoding = 'utf-8')
test_data = pd.read_csv('https://raw.githubusercontent.com/jsAiyaya/JSC270_Data_Science_1/main/covid-tweets-test.csv', encoding = 'utf-8')

In [ ]:
train_data = train_data.drop(['Unnamed: 0'], axis = 1)
test_data = test_data.drop(['Unnamed: 0'], axis = 1)

In [ ]:
test_data.head(5)

,OriginalTweet,Sentiment
0,TRENDING: New Yorkers encounter empty supermar...,0
1,When I couldn't find hand sanitizer at Fred Me...,2
2,Find out how you can protect yourself and love...,2
3,#Panic buying hits #NewYork City as anxious sh...,0
4,#toiletpaper #dunnypaper #coronavirus #coronav...,1


In [ ]:
train_data.drop(train_data.index[(train_data['Sentiment'] != '0') & (train_data['Sentiment'] != '1') & (train_data['Sentiment'] != '2')], inplace= True)
test_data.drop(test_data.index[(test_data['Sentiment'] != 0) & (test_data['Sentiment'] != 1) & (test_data['Sentiment'] != 2)], inplace= True)
train_data['Sentiment'] = train_data['Sentiment'].astype('int')

# **Part 1**

### **(A)**

In [ ]:
count_table = train_data.groupby('Sentiment').count()
count_table.columns = ["count"]
print(count_table, "\n")

total_num = len(train_data)
prop_table = count_table / total_num
prop_table.columns = ["proportion"]
print(prop_table)

           count
Sentiment       
0          15397
1           7712
2          18042 

           proportion
Sentiment            
0            0.374159
1            0.187407
2            0.438434


### **(B)**

In [ ]:
def tokenize(df):
  df['tokens'] = df['OriginalTweet'].apply(nltk.word_tokenize)
tokenize(train_data)
tokenize(test_data)

### **(C)**

In [ ]:
# remove tokens beginning with http
def remove_url(df):
  tokens_no_url = []
  for row in df['tokens']:
    tokens_no_url.append([re.sub('http.+', '', t) for t in row])
  df['tokens'] = tokens_no_url

remove_url(train_data)
remove_url(test_data)

### **(D)**

In [ ]:
def remove_punct(df):
  tokens_no_punct = []
  for row in df['tokens']:
    tokens_no_punct.append([re.sub('[^\w\s]', '', t) for t in row])
  df['tokens'] = tokens_no_punct

remove_punct(train_data)
remove_punct(test_data)

In [ ]:
def lowercase(df):
  lowercase_tokens = []
  for row in df['tokens']:
    lowercase_tokens.append([t.lower() for t in row])
  df['tokens'] = lowercase_tokens

lowercase(train_data)
lowercase(test_data)

### **(E)**

In [ ]:
# We use the Porter stemmer
stemmer = PorterStemmer()

def stem(df):
  stemmed_tokens = []
  for row in train_data['tokens']:
    stemmed_tokens.append([stemmer.stem(t) for t in row])
  train_data['tokens'] = stemmed_tokens

stem(train_data)
stem(test_data)

### **(F)**

In [ ]:
sw = stopwords.words('english')[:100]

def remove_stopwords(df):
  tokens_no_sw = []
  for row in df['tokens']:
    tokens_no_sw.append([w for w in row if w not in sw])
  df['tokens'] = tokens_no_sw

remove_stopwords(train_data)
remove_stopwords(test_data)

### **(G)**

In [ ]:
# Separate labels from features, converting to numpy arrays
X_train, y_train = train_data['tokens'].to_numpy(), train_data['Sentiment'].to_numpy()
X_test, y_test = test_data['tokens'].to_numpy(), test_data['Sentiment'].to_numpy()

# Since we've already done some steps, we will use a dummy function to override these components.
def override_fcn(doc):
  # We expect a list of tokens as input
  return doc

# Count Vectorizer
count_vec = CountVectorizer(
    analyzer='word',
    tokenizer= override_fcn,
    preprocessor= override_fcn,
    token_pattern= None,
    max_features = 1000)

counts_train = count_vec.fit_transform(X_train)
counts_test = count_vec.fit_transform(X_test)
print("train data:")
print(counts_train.toarray())
print("\n test data:")
print(counts_test.toarray())

train data:
[[9 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [4 0 0 ... 0 0 0]
 ...
 [7 0 0 ... 0 0 0]
 [4 0 0 ... 0 0 0]
 [5 0 0 ... 0 0 0]]

 test data:
[[13  0  0 ...  0  0  0]
 [12  0  0 ...  0  0  0]
 [ 3  0  0 ...  0  0  0]
 ...
 [ 9  0  0 ...  0  0  0]
 [ 6  0  0 ...  0  0  0]
 [ 8  0  0 ...  0  0  0]]
